## 1.Settings

### 1) Import required libraries

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
import unidecode
import string
import random
import re
import time, math

### 2) Hyperparameter

In [4]:
num_epochs = 10000
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150
hidden_size = 100
batch_size =1
num_layers = 2
lr = 0.002

## 2. Data

### 1) Prepare characters

In [5]:

all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


### 2) Get text data

In [6]:
file = unidecode.unidecode(open('./data/shakespeare.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


## 3. Functions for text processing

### 1) Random Chunk

In [7]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

bridal-day,
Even then that sunshine brew'd a shower for him,
That wash'd his father's fortunes forth of France,
And heap'd sedition on his crown at home.
For what hath broach'd this tumult but thy prid


### 2) Character to tensor

In [10]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


### 3) Chunk into input & label

In [11]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

## Model & Optimizer

### 1) Model

In [12]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.GRU(embedding_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))
        out,hidden = self.rnn(out,hidden)
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.num_layers, batch_size, hidden_size))
        return hidden
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers=2)

In [13]:
inp = char_tensor("A")
print(inp)
hidden = model.init_hidden()
print(hidden.size())

out,hidden = model(inp,hidden)
print(out.size())

tensor([36])
torch.Size([2, 1, 100])
torch.Size([1, 100])


### 2) Loss & Optimizer

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

### 3) Test funtion

In [15]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

## 4. Train

In [16]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden()

    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j]
        y,hidden = model(x,hidden)
        loss += loss_func(y,y_.view(-1))

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")


 tensor(4.5876, grad_fn=<DivBackward0>) 

6\14l:l.q_HuzTt*;AmE?OYF$_j`S/Kc<.]voyU!431;,tLzGKjN<\j'Vi1q]-^
t{NocL+k9Fn,d3/Y1ShUjs/X}U4v;1B*4}JyF3s8\2EoG
%!;<2$?}oD8:q^,M)ke;to45wEBi2p1?)[:>r}TnyQ%vp_99r=JhvN



 tensor(2.5535, grad_fn=<DivBackward0>) 

bb.k
GFat ethotegotede wont sir and, heit ats arce anen wo thar ro berelly bery the th pelf I to si:
Than sorr:
AKsors, yoremree thor ou my mas thee ron ome alsNe to visan mo ond ure the onse mass? oTe



 tensor(2.2580, grad_fn=<DivBackward0>) 

bfir motlave it bepad that
Merarg har you be ae.

Fuind ame wenerend shamlasit auly his the mere we, sire the li I the the pume, hour moreit hit the palle tor.

IN:
Khaure no hot therrfe sonce at ay hi



 tensor(2.1318, grad_fn=<DivBackward0>) 

bckzis pon we the fislere sovoucn howe, sond mome.

LUREI:
Me, the gory tiy lice my wo ther courd shoy wher and courwathe were mill ethe ot coth wer, ant and to sere turt may thou; Lort usrer
Witd hich



 tensor(2.1689, grad_fn=<DivBackward0>) 



A ficks of as not ficess I thou doing him nemble
As the good sifted him enthing hand.

Tood Clarecthed:
How that I say with day:
And him fortiop is me?

BENVOLIO:
Take ent



 tensor(1.5828, grad_fn=<DivBackward0>) 

bern art trie;
But suep from mispeak, were of you shall be patient
The mancl of thus as of of your pence peaces,
Or being brains it his hear's arm souls
For both all give of heart,
To the piirant
To th



 tensor(1.4114, grad_fn=<DivBackward0>) 

bhe too thought
Tonded of the no mascranded: then an wet.

GLOUCESTER:
Hath with let which like hearts. The ears
Shall heave pity bear we well to strue 'twere speak her sent
Hanry, for any old to men a



 tensor(1.6306, grad_fn=<DivBackward0>) 

ben, store mest alongs, 'tis love.

GLOUCESTER:
What son, the muntio my more which the trail might;
What should not the now them, by what love and muker:
Commore you the for whood,
That say to be her d



 tensor(1.1893, grad_fn=<DivBackward0>) 

best not stand thee;
The hears and by sang

To love men to a world,
Which you, Englank of sad go and ibour death.





 tensor(1.5475, grad_fn=<DivBackward0>) 

besting complaxed;
Let the poor, that doubtles, and finds thou, and the crown,
So go, and supposter to thinkly breathy like true?

KING RICHARD III:
I see a commond and blacks; ever noble dere thee,
th



 tensor(1.3895, grad_fn=<DivBackward0>) 

ble,
And who dark but here,--

The Cancillent means Clond and compace,
Who stands, and timent;
And to know thyseld him of a things
Upon the trenched, and learnial begreat, be the boude,
That I a mint,




 tensor(1.4030, grad_fn=<DivBackward0>) 

brace at him breast not little,
Could throses, it be a more a stand,
Time of a trice you his be vised of gracely
That he be send to thing us doth, a litchouse.

RICHARD:
Say fair day he have thou had o



 tensor(1.5064, grad_fn=<DivBackward0>) 

by he.

POMPEY:
And it is poolence, and laig'd hath else of the state so make's.

AMILINA:
Master her men thy stone; for the good so ews,
And 